In [1]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.gaussian_process.kernels import RBF

import numpy as np
import matplotlib.pyplot as plt

from cv_svm import SVM_smooth
from kernel_svm import SVM_smooth_kernel
from sampler import sample_from_logreg

from sklearn.preprocessing import normalize

import time

import matplotlib

In [9]:
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [3]:
sigma = 1e-5
lbd = 1
gamma = 0.1
length_scale = np.sqrt(1/(2*gamma))
experiment_sizes = np.linspace(10, 550, 8)

time_taken = {
    "IACV": np.ones(len(experiment_sizes)),
    "true": np.ones(len(experiment_sizes))
}

err_approx = []
err_cv = []

In [4]:
def Phi_m(v):
    return (1 + v / np.sqrt(1 + v**2)) / 2

def Phi_m_prime(v):
    return 1 / 2 * 1 / (np.sqrt(v**2 + 1) ** 3)

def phi_m(v):
    return 1 / (2 * np.sqrt(1 + v**3))

def Psi_m(alpha, sigma):
    return (
        Phi_m((1 - alpha) / sigma) * (1 - alpha)
        + phi_m((1 - alpha) / sigma) * sigma
    )

def loss(w, X, y, sigma):
    return Psi_m(y * (X @ w), sigma)


def SSVM_objective(w, X, y, sigma, lbd):
    return 1 / 2 * np.linalg.norm(w)**2 * lbd + loss(w, X, y, sigma)


def SVM_objective(w, X, y, C):
    return 1 / 2 * np.linalg.norm(w)**2 + C * max(0, 1 - y * (X @ w))

In [5]:
# TODO, technically incorrect but who cares
for i, n in enumerate(experiment_sizes):
    n = int(n)
    X, _, y = sample_from_logreg(n=n, p=20)
    gram = RBF(length_scale)(X, X)

    max_iter = int(n * 1.25)
    print(f"running experiments for size {n} | max_iter = {max_iter}")
    c_val = 1/n
    clf = SVC(kernel='rbf', gamma=gamma, C=c_val)
    true_start = time.time()
    true_params = []
    true_loss = []
    for j in range(n):
        X_temp = np.delete(X, (j), axis=0)
        y_temp = np.delete(y, (j), axis=0)
        coef_z = np.zeros(n)
        clf.fit(X_temp, y_temp)
        coef_z[clf.support_] = clf.dual_coef_[0]
        true_params.append(coef_z)
        true_loss.append(SVM_objective(coef_z, gram[j, :], y[j], c_val))
    true_end = time.time()
    print(f"\t true CV: {true_end - true_start}s")
    
    clf = SVM_smooth_kernel(sigma=sigma, lbd=lbd, kernel=RBF(length_scale))
    approx_start = time.time()
    clf.fit(X, y, thresh=1e-3, n_iter=max_iter, cv=False, approx_cv=True)
    approx_end = time.time()
    print(f"\t approx CV: {approx_end - approx_start}s")

    approx_loss = []
    for j in range(n):
        approx_loss.append(SSVM_objective(clf.approx_cv_obj.iterates[j], gram[j, :], y[j], sigma, lbd))

    time_taken["true"][i] = true_end - true_start
    time_taken["IACV"][i] = approx_end - approx_start

    true_params_norm = normalize(true_params, axis=1)
    approx_params_norm = normalize(clf.approx_cv_obj.iterates.T, axis=1)
    err_approx.append(np.linalg.norm(true_params_norm - approx_params_norm, 2, axis=1).mean())
    
    true_loss_norm = normalize(np.asarray(true_loss).reshape(-1, 1), axis=0)
    approx_loss_norm = normalize(np.asarray(approx_loss).reshape(-1, 1), axis=0)
    err_cv.append(np.abs(true_loss_norm - approx_loss_norm).mean())
    print(f"\t err approx {err_approx[-1]} | err cv {err_cv[-1]}")

running experiments for size 10 | max_iter = 12
	 true CV: 0.00418400764465332s
	 approx CV: 0.5723116397857666s
	 err approx 1.4325933644605107 | err cv 0.01759728860548459
running experiments for size 87 | max_iter = 108
	 true CV: 0.05027937889099121s
	 approx CV: 0.5454885959625244s
	 err approx 1.4170434259434204 | err cv 0.0004942058868850663
running experiments for size 164 | max_iter = 205
	 true CV: 0.1879284381866455s
	 approx CV: 1.0302565097808838s
	 err approx 1.4154841699060456 | err cv 0.0003867414508754939
running experiments for size 241 | max_iter = 301
	 true CV: 0.536768913269043s
	 approx CV: 2.5979039669036865s
	 err approx 1.4152041914085962 | err cv 0.0003300114438845315
running experiments for size 318 | max_iter = 397
	 true CV: 1.038933277130127s
	 approx CV: 6.325984954833984s
	 err approx 1.415295350905124 | err cv 0.00033463958987868867
running experiments for size 395 | max_iter = 493
	 true CV: 1.9239602088928223s
	 approx CV: 13.174010276794434s
	 err a

In [15]:
fig, ax = plt.subplots()
ax.plot(experiment_sizes, time_taken["IACV"], label="IACV", c='black')
ax.plot(experiment_sizes, time_taken["true"], label="True", c='green')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Time taken (s)")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
ax.legend()
plt.show()
plt.savefig('kernel_true_iacv_benchmark.pgf', bbox_inches='tight', pad_inches=0)

/tmp/ipykernel_3618/2535686745.py:11: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
fig, ax = plt.subplots()
ax.plot(experiment_sizes, err_approx, label="IACV", c='black')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Err Approx")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
plt.savefig('kernel_iacv_err_approx_benchmark.pgf', bbox_inches='tight', pad_inches=0)

/tmp/ipykernel_3618/2779509920.py:9: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
fig, ax = plt.subplots()
ax.plot(experiment_sizes, err_cv, label="IACV", c='black')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Err CV")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
plt.savefig('kernel_iacv_err_cv_benchmark.pgf', bbox_inches='tight', pad_inches=0)

/tmp/ipykernel_3618/229302137.py:9: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [16]:
length_scale

2.23606797749979